<a href="https://colab.research.google.com/github/softmurata/AudioML/blob/main/style_bert_vits.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installation

In [ ]:
!git clone https://github.com/litagin02/Style-Bert-VITS2.git
%cd Style-Bert-VITS2/
!pip install -r requirements.txt
!apt install libcublas11
!python initialize.py

Cloning into 'Style-Bert-VITS2'...
remote: Enumerating objects: 3139, done.
remote: Counting objects: 100% (1504/1504), done.
remote: Compressing objects: 100% (384/384), done.
remote: Total 3139 (delta 1226), reused 1270 (delta 1118), pack-reused 1635
Receiving objects: 100% (3139/3139), 10.69 MiB | 15.40 MiB/s, done.
Resolving deltas: 100% (1943/1943), done.
/content/Style-Bert-VITS2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 939.4/939.4 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.0/225.0 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 23.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 16.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 55.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 49.7 MB/s eta 0:00:00
  Pre

In [1]:
%cd /content/Style-Bert-VITS2/
# 学習に必要なファイルや途中経過が保存されるディレクトリ
dataset_root = "/content/Style-Bert-VITS2/Data"

# 学習結果（音声合成に必要なファイルたち）が保存されるディレクトリ
assets_root = "/content/Style-Bert-VITS2/model_assets"

import yaml


with open("configs/paths.yml", "w", encoding="utf-8") as f:
    yaml.dump({"dataset_root": dataset_root, "assets_root": assets_root}, f)

/content/Style-Bert-VITS2


In [ ]:
# 元となる音声ファイル（wav形式）を入れるディレクトリ
input_dir = "/content/Style-Bert-VITS2/inputs"
# モデル名（話者名）を入力
model_name = "anya"

!python slice.py -i {input_dir} -o {dataset_root}/{model_name}/raw
!python transcribe.py -i {dataset_root}/{model_name}/raw -o {dataset_root}/{model_name}/esd.list --speaker_name {model_name} --compute_type float16

学習データを前もって用意

In [6]:
import os
os.makedirs("/content/Style-Bert-VITS2/Data/anya/raw")

In [7]:
!cp -r /content/Style-Bert-VITS2/inputs/* /content/Style-Bert-VITS2/Data/anya/raw/

In [9]:
# ファイルを読み込む
with open(dataset_root + "/anya/anya.txt", "r", encoding="utf-8") as file:
    lines = file.readlines()

# コンバート
result = []
for line in lines:
    strs = line.split("|")
    filename = strs[0].split("/")[-1]
    script = strs[2]
    result.append(f"/content/Style-Bert-VITS2/Data/anya/raw/{filename}|anya|JP|{script}")

# 変更をファイルに保存
with open(dataset_root + "/anya/esd.list", "w", encoding="utf-8") as file:
    file.writelines(result)

Preprocess

In [ ]:
%cd /content/Style-Bert-VITS2/
# 上でつけたフォルダの名前`Data/{model_name}/`
model_name = "anya"

# 学習のバッチサイズ。VRAMのはみ出具合に応じて調整してください。
batch_size = 4

# 学習のエポック数（データセットを合計何周するか）。
# 100ぐらいで十分かもしれませんが、もっと多くやると質が上がるのかもしれません。
epochs = 100

# 保存頻度。何ステップごとにモデルを保存するか。分からなければデフォルトのままで。
save_every_steps = 1000

# 音声ファイルの音量を正規化するかどうか。`True`もしくは`False`
normalize = False

# 音声ファイルの開始・終了にある無音区間を削除するかどうか
trim = False


from webui_train import preprocess_all

preprocess_all(
    model_name=model_name,
    batch_size=batch_size,
    epochs=epochs,
    save_every_steps=save_every_steps,
    bf16_run=False,
    num_processes=2,
    normalize=normalize,
    trim=trim,
)

Training

In [ ]:
%cd /content/Style-Bert-VITS2/
# 上でつけたモデル名を入力。学習を途中からする場合はきちんとモデルが保存されているフォルダ名を入力。
model_name = "anya"


import yaml
from webui_train import get_path

dataset_path, _, _, _, config_path = get_path(model_name)

with open("default_config.yml", "r", encoding="utf-8") as f:
    yml_data = yaml.safe_load(f)
yml_data["model_name"] = model_name
with open("config.yml", "w", encoding="utf-8") as f:
    yaml.dump(yml_data, f, allow_unicode=True)

!python train_ms.py --config {config_path} --model {dataset_path} --assets_root {assets_root}

Inference with webapp

In [ ]:
%cd /content/Style-Bert-VITS2/
!python app.py --share --dir {assets_root}